In [1]:
# Google API Key
from config import gkey 

In [2]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

In [3]:
df = pd.read_csv("airport.csv")
df

,City,State
0,New York City,New York
1,Los Angeles,California
2,Chicago,Illinois
3,Houston,Texas
4,Philadelphia,Pennsylvania
...,...,...
95,Fremont,California
96,Baton Rouge,Louisiana
97,Richmond,Virginia
98,Boise,Idaho


In [4]:
df["Lat"] = ""
df["Lng"] = ""
df["Airport Name"] = ""
df["Airport Address"] = ""
df["Airport Rating"] = ""
df

,City,State,Lat,Lng,Airport Name,Airport Address,Airport Rating
0,New York City,New York,,,,,
1,Los Angeles,California,,,,,
2,Chicago,Illinois,,,,,
3,Houston,Texas,,,,,
4,Philadelphia,Pennsylvania,,,,,
...,...,...,...,...,...,...,...
95,Fremont,California,,,,,
96,Baton Rouge,Louisiana,,,,,
97,Richmond,Virginia,,,,,
98,Boise,Idaho,,,,,


In [6]:
params = {"key": gkey}

for index, row in df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    city = row["City"]
    state = row["State"]
    
    params["address"] = f"{city},{state}"
    
    cities_lat_lng = requests.get(base_url, params=params)
    
    cities_lat_lng = cities_lat_lng.json()

    df.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    df.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
df.tail()   

,City,State,Lat,Lng,Airport Name,Airport Address,Airport Rating
95,Fremont,California,37.5485,-121.989,,,
96,Baton Rouge,Louisiana,30.4515,-91.1871,,,
97,Richmond,Virginia,37.5407,-77.436,,,
98,Boise,Idaho,43.615,-116.202,,,
99,San Bernardino,California,34.1083,-117.29,,,


In [7]:
params = {
    "radius": 50000,
    "types": "airport",
    "keyword": "international airport",
    "key": gkey
}

for index, row in df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    
    try: 
        df.loc[index, "Airport Name"] = name_address["results"][0]["name"]
        df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError): 
        print("Missing field/result...skipping")        

Missing field/result...skipping


In [8]:
df

,City,State,Lat,Lng,Airport Name,Airport Address,Airport Rating
0,New York City,New York,40.7128,-74.006,John F. Kennedy International Airport,Queens,3.8
1,Los Angeles,California,34.0522,-118.244,Los Angeles International Airport,"1 World Way, Los Angeles",3.7
2,Chicago,Illinois,41.8781,-87.6298,O'Hare International Airport,"10000 W O'Hare Ave, Chicago",3.8
3,Houston,Texas,29.7604,-95.3698,Dan Jones International Airport,"16434 Kitzman St, Cypress",3.9
4,Philadelphia,Pennsylvania,39.9526,-75.1652,Philadelphia International Airport,"8000 Essington Ave, Philadelphia",3.6
...,...,...,...,...,...,...,...
95,Fremont,California,37.5485,-121.989,San Francisco International Airport,San Francisco,4
96,Baton Rouge,Louisiana,30.4515,-91.1871,Baton Rouge Metropolitan Airport,"9430 Jackie Cochran Dr, Baton Rouge",4.1
97,Richmond,Virginia,37.5407,-77.436,Richmond International Airport,"1 Richard E Byrd Terminal Dr, Richmond",4.2
98,Boise,Idaho,43.615,-116.202,Boise Airport,"3201 W Airport Way #1000, Boise",4.5


In [9]:
df.to_csv("output.csv")